In [4]:
import numpy as np
import gurobipy as grb
import pandas as pd
from collections import defaultdict

In [2]:


workers = {'A', 'B', 'C'}

tasks = set(range(1, 11))

c = defaultdict(lambda: 1000, {
    ('A',  2):  7,
    ('A',  3):  3,
    ('A',  6): 18,
    ('A',  7): 13,
    ('A',  8):  6,
    ('A', 10):  9,
    ('B',  1): 12,
    ('B',  2):  5,
    ('B',  4): 12,
    ('B',  5):  4,
    ('B',  6): 22,
    ('B',  8): 17,
    ('B',  9): 13,
    ('C',  1): 18,
    ('C',  3):  6,
    ('C',  4):  8,
    ('C',  5): 10,
    ('C',  7): 19,
    ('C',  9):  8,
    ('C', 10): 15,
})

max_hours = 40



In [5]:
model = grb.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-02


In [6]:
x = model.addVars(workers, tasks, vtype='C', lb=0, ub=1)
model.update()
print(x)

{('B', 1): <gurobi.Var C0>, ('B', 2): <gurobi.Var C1>, ('B', 3): <gurobi.Var C2>, ('B', 4): <gurobi.Var C3>, ('B', 5): <gurobi.Var C4>, ('B', 6): <gurobi.Var C5>, ('B', 7): <gurobi.Var C6>, ('B', 8): <gurobi.Var C7>, ('B', 9): <gurobi.Var C8>, ('B', 10): <gurobi.Var C9>, ('A', 1): <gurobi.Var C10>, ('A', 2): <gurobi.Var C11>, ('A', 3): <gurobi.Var C12>, ('A', 4): <gurobi.Var C13>, ('A', 5): <gurobi.Var C14>, ('A', 6): <gurobi.Var C15>, ('A', 7): <gurobi.Var C16>, ('A', 8): <gurobi.Var C17>, ('A', 9): <gurobi.Var C18>, ('A', 10): <gurobi.Var C19>, ('C', 1): <gurobi.Var C20>, ('C', 2): <gurobi.Var C21>, ('C', 3): <gurobi.Var C22>, ('C', 4): <gurobi.Var C23>, ('C', 5): <gurobi.Var C24>, ('C', 6): <gurobi.Var C25>, ('C', 7): <gurobi.Var C26>, ('C', 8): <gurobi.Var C27>, ('C', 9): <gurobi.Var C28>, ('C', 10): <gurobi.Var C29>}


In [7]:
expr = sum(c[w, t] * x[w, t]
           for w in workers for t in tasks)
model.setObjective(sense=grb.GRB.MINIMIZE, expr=expr)


In [9]:
for t in tasks:
    lhs = sum(x[w, t] for w in workers)
    rhs = 1
    model.addConstr(lhs == rhs)

In [10]:
for w in workers:
    lhs = sum(c[w, t] * x[w, t] for t in tasks)
    rhs = max_hours
    model.addConstr(lhs <= rhs)

In [11]:
model.update()

In [12]:
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 23 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x91935b52
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 10 rows and 0 columns
Presolve time: 0.00s
Presolved: 13 rows, 30 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8000000e+01   1.200000e+01   0.000000e+00      0s
      10    8.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds (0.00 work units)
Optimal objective  8.800000000e+01


In [13]:
df = pd.DataFrame(index=pd.MultiIndex.from_tuples(x, names=['w', 't']))
df['x'] = [x[k].x for k in df.index]
df['c'] = [c[k] for k in df.index]

In [14]:
(df['c'] * df['x']).unstack('t')

t,1,2,3,4,5,6,7,8,9,10
w,,,,,,,,,,
A,0.0,0.0,3.0,0.0,0.0,9.0,13.0,6.0,0.0,9.0
B,12.0,5.0,0.0,0.0,4.0,11.0,0.0,0.0,0.0,0.0
C,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,8.0,0.0


In [15]:
(df['c'] * df['x']).groupby('w').sum().to_frame()

,0
w,
A,40.0
B,32.0
C,16.0


In [16]:
df['x'].groupby('t').sum().to_frame().T

t,1,2,3,4,5,6,7,8,9,10
x,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
